<a href="https://www.kaggle.com/code/aisuko/fine-tuning-mistral-7b-with-dpo?scriptVersionId=163920200" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

**Note: All the images are from the Credits section or the internet. And this one was not reviewed completely because no enough computing resources, and bf16 type is not supported by current env.**

Pre-trained LLMs can only perform next-token prediction, making them unable to answer questions. This is why these base models are then fine-tuned on pairs of instructions and anwers to act as helpful assistants. However, this process can still be flawed: fine-tuned LLMs can be biased, toxic, harmful, etc. This is where Reinforcement Learning from RLHF(Reinforcement Learning from Human Feedback) comes into play.

RLHF provides different answers to the LLM, which are ranked according to a desired behavior(helpfulness, toxicity, etc). The model learns to output the best answer among these candidates, hence mimicking the behavior we want to instill. Often seen as a way to censor models, this process has recently become popular for improving performance, as shown in [neural-chat-7b-v3-1](https://huggingface.co/Intel/neural-chat-7b-v3-1).

We are going to fine-tune OpenHermes-2.5 using a RLHF-like technique: **Direct Preference Optimization(DPO)**.

In [1]:
%%capture
!pip install transformers==4.36.2
!pip install datasets==2.15.0
!pip install peft==0.7.1
!pip install bitsandbytes==0.41.3
!pip install trl==0.7.7

In [2]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "Fine-tune-models"
os.environ["WANDB_NOTES"] = "Fine tune model distilbert base uncased"
os.environ["WANDB_NAME"] = "ft-openhermes-25-mistral-7b-irca-dpo-pairs"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Preparing datasets

We are prefer the **Preference datasets**. Typically consist of a collection of answers that are ranked by humans. This ranking is seential, as the RLHF process fine-tunes LLMs to output the preferred answer.

In [3]:
from datasets import load_dataset

train_ds=load_dataset("Intel/orca_dpo_pairs")["train"]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

The structure of the dataset is straightforward: for each row, there is one chosen(preferred) answer, and one rejected answer. The goal of RLHF is to guide the model to output the preferred answer.

## Preference datasets

They are notoriously costly and difficult to make, as they require collecting manual feedback from humans. This feedback is also subjective and can easily be biased toward confident(but wrong) answers or contradict itself (different annotators have different values). Over time, several solutions have been proposed to tackle these issues, such as replacing human feedback with AI feedback [RLAIF](https://arxiv.org/abs/2212.08073).

These datasets also tend to be a lot smaller than fine-tuninf datasets. To illustrate this, the excellent [neural-chat-7b-v3-1](https://huggingface.co/Intel/neural-chat-7b-v3-1) uses 518k samples for fine-tuning [Open-Orca/SlimOrca](https://huggingface.co/datasets/Open-Orca/SlimOrca) but only 12.9k samples for RLHF [Intel/orca_dpo_pairs](https://huggingface.co/datasets/Intel/orca_dpo_pairs). In this case, the authors generated answers with GPT-4/3.5 to create the preferred answers, and with [Llama 2 13b chat](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) to create the rejected responses. It's a smart way to bypass human feedback and only rely on models with different levels of performance.

# Direct Preference Optimization

While the concept of RLHF has been used in robotics for a long time, it was popularized for LLMs in [Fine-tuning Language Models from Human Preferences](https://arxiv.org/pdf/1909.08593.pdf). In this paper, the authors present a framework where a reward model is trained to approximate human feedback. This reward model is then used to optimize the fine-tuned model's policy using the [Proximal Policy Optimization algorithm](https://arxiv.org/abs/1707.06347).

<div style="text-align: center"><img src="https://files.mastodon.social/media_attachments/files/111/707/466/912/510/214/original/5f3fa6b5a0b186a3.webp" width="60%" heigh="60%" alt="proximal policy optimization"></div>


The Core concept of PPO revolves around making smaller, incremental updates to the policy, as larger updates can lead to instability or suboptimal solutions. From experience, this technique is unfortunately still unstable(loss diverges), difficult to reproduce(numerous hyperparameters, sensitive to random seeds), and computationally expensive.

This is where Direct Preference Optimization(DPO) comes into play. DPO simplifies control by treating the task as a classification problem. Concretely, it uses two models: the **trained model(or policy model)** and a copy of it called the **reference model**. During training, the goal is to make sure the trained model outputs higher probabilities for preferred answers than the reference model. Conversely, we also want it output lower probabilities for rejected answers. It means we're penalizing the LLM for bad answers and rewarding it for good ones.

<div style="text-align: center"><img src="https://files.mastodon.social/media_attachments/files/111/707/528/109/748/161/original/7c1ef620f48dd4a6.webp" width="60%" heigh="60%" alt="direct preference optimization"></div>

By using the LLM itself as a reward model and employing binary cross-entropy objectives, DPO efficiently aligns the model's outputs with human preferences without the need for extensive sampling, reward model fitting, or intricate hyperparameters adjustments. It results in a more stable, more efficient, and computationally less demanding process.

# Formatting the data

Here we are going to fine-tune [OpenHermes-2.5-Mistral-7B](https://huggingface.co/teknium/OpenHermes-2.5-Mistral-7B) which is a Mistral-7b model that was only supervised fine-tuned. To this end, we will use the [Intel/orca_dpo_pairs](https://huggingface.co/datasets/Intel/orca_dpo_pairs) dataset to align our model and improve this performance.

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, PeftModel, get_peft_model
from trl import DPOTrainer

In [5]:
model_name="teknium/OpenHermes-2.5-Mistral-7B"

OpenHermes-2.5-Mistral-7B uses a specific chat template, called [ChatML](https://huggingface.co/docs/transformers/chat_templating). Here is an example of conversation formatted with this template:

```
<|im_start|>system
You are a helpful chatbot that will do its best not to say anything so stupid that people tweet about it.<|im_end|>
<|im_start|>user
How are you?<|im_end|>
<|im_start|>assistant
I'm doing great!<|im_end|>
```

Each of sentence which includes a role, like "system, "user" or "assistant", and it appends special tokens at the beginning and the end of the sentence(<|im_start|> and <|im_end|>). These two are used to separate different sentences. Moreover, DPOTrainer also requires a specific format with three columns: prompt, chosen and rejected.

We will simply concatenate the system and question columns to the prompt column. We will also map the chatgpt column to "chosen" and llama2-13b-chat to "rejected". To format the dataset in a reliable way, we will use the tokenizer's `apply_chat_template()` function, which already use ChatML.

In [6]:
def chatml_format(example):
    # format system
    if len(example['system'])>0:
        message ={"role":"system","content":example['system']}
        system=tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system=""
    
    #format instruction
    message={"role":"user","content":example['question']}
    prompt=tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
    
    # format chosen answer
    chosen =example['chosen']+"<im_end>\n"
    
    # format rejected answer
    rejected = example['rejected']+"<im_end>\n"
    
    return {
        "prompt": system+prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# we have load datasets in above section

# save columns
original_columns=train_ds.column_names

# tokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="left"

#format dataset
train_dataset=train_ds.map(
    function=chatml_format,
    remove_columns=original_columns
)

tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/12859 [00:00<?, ? examples/s]

We can see the one of the example in the formated dataset.

In [7]:
train_dataset[0]

{'chosen': '[\n  ["AFC Ajax (amateurs)", "has ground", "Sportpark De Toekomst"],\n  ["Ajax Youth Academy", "plays at", "Sportpark De Toekomst"]\n]<im_end>\n',
 'rejected': " Sure, I'd be happy to help! Here are the RDF triplets for the input sentence:\n\n[AFC Ajax (amateurs), hasGround, Sportpark De Toekomst]\n[Ajax Youth Academy, playsAt, Sportpark De Toekomst]\n\nExplanation:\n\n* AFC Ajax (amateurs) is the subject of the first triplet, and hasGround is the predicate that describes the relationship between AFC Ajax (amateurs) and Sportpark De Toekomst.\n* Ajax Youth Academy is the subject of the second triplet, and playsAt is the predicate that describes the relationship between Ajax Youth Academy and Sportpark De Toekomst.\n\nNote that there may be other possible RDF triplets that could be derived from the input sentence, but the above triplets capture the main relationships present in the sentence.<im_end>\n",
 'prompt': "<|im_start|>user\nYou will be given a definition of a task f

# Training the model with DPO

We define the LoRA configurations to train the model. We set the rank value to be equal to the `lora_alpha`, which is unusual(2*`r` as a rule of thumb). We also target all the linear modules with adapters.

In [8]:
from transformers import BitsAndBytesConfig
import torch

# LoRA configuration
peft_config=LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj','gate_proj','v_proj','up_proj','q_proj','o_proj','down_proj']
)

bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
#     llm_int8_enable_fp32_cpu_offload=True,
)

## Quantize the model

We're now ready to load the model we want to fine-tune with DPO. In this case, two models are required: the model to fine-tune as well as the reference model. This is mostly for the sake of readability, as the `DPOTrainer` object automatically creates a ference model if none is provided.

In [9]:
# model to fine-tune

model=AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)

model

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [10]:
model.config.use_cache=False
model=get_peft_model(model, peft_config)
model.get_memory_footprint()

4719165440

In [11]:
model.config

MistralConfig {
  "_name_or_path": "teknium/OpenHermes-2.5-Mistral-7B",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4

In [12]:
# Reference model
ref_model=AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)
ref_model=get_peft_model(ref_model, peft_config)
ref_model.get_memory_footprint()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

4719165440

In [13]:
ref_model.config

MistralConfig {
  "_name_or_path": "teknium/OpenHermes-2.5-Mistral-7B",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4

The final step consists of providing all the hyperparameters to `TrainingArguments` and `DPOTrainer`:
* Among them, the `beta` parameter is unique to DPO since it controls the divergence from the initial policy(0.1 is a typical value for it).

In [ ]:
training_args=TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    # key word argumnts to be passed to the gradient_checkingpointing_enable method
    gradient_checkpointing=True,
    gradient_accumulation_steps=5,
    remove_unused_columns=False,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    # the total number of training steps to perform
    max_steps=100,
    save_strategy="no",
    logging_steps=100,
    output_dir=os.getenv("WANDB_NAME"),
    optim="paged_adamw_32bit",
    bf16=False, # Doesn;t support in Kaggle environment
    fp16=True,
    # Number of steps used for a linear warmup from 0 to learning_rate
    warmup_steps=50,
    run_name=os.getenv("WANDB_NAME"),
    report_to="wandb"
)

dpo_trainer=DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

# Kaggle env does not have enough Memory to doing training, it will restarted and cause error.
dpo_trainer.train()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


# Saving the merged model

In [ ]:
dpo_trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

# Flush memory
del dpo_trainer, model, ref_model
gc.collect()
torch.cuda.empty_cache()

# Reload model in FP16(instead of NF4)
base_model=AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_type=torch.float16,
)

tokenizer=AutoTokenizer.from_pretrained(model_name)

# merge base model with the adapter
model=PeftModel.from_pretrained(base_model, "final_checkpoint")
model=model.merge_and_unload()

# save model and tokenizer
model.save_pretrained(os.getenv("WANDB_NAME"))
tokenizer.save_pretrained(os.getenv("WANDB_NAME"))


model.push_to_hub(os.getenv("WANDB_NAME"))
tokenizer.push_to_hub(os.getenv("WANDB_NAME"))

# Checking the model performs

Excepting run the model locally, we can also leverage the [lm-evaluation-harness](https://github.com/EleutherAI/lm-evaluation-harness) to evaluate it. As the process is quite resource-intensive, we can also directly submit it for evaluation on the [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard).

In [ ]:
# format prompt
message=[
    {"role":"system", "content":"Teh weather is important."},
    {"role":"user","content":"Does Melborune raining today?"}
]

tokenizer=AutoTokenizer.from_pretrained(os.getenv("WANDB_NAME"))
prompt=tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)

pipe=transformers.pipeline(
    "text-generation",
    model=os.getenv("WANDB_NAME"),
    tokenizer=tokenizer
)

# generate text
sequences=pipe(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)

sequences[0]['generated_text']

# Credits

* https://towardsdatascience.com/fine-tune-a-mistral-7b-model-with-direct-preference-optimization-708042745aac